In [125]:
import numpy as np
import math

def part_1_solution(raw_input):
    chemicals = parse_raw_input(raw_input)
    basic_chemicals = get_basic_chemicals(chemicals)
    # assuming quantity of fuel is 1
    ore_cnt = 0
    for basic_chemical in basic_chemicals:
        precise_quantity_needed = get_symbol_quantity("FUEL", basic_chemical, chemicals)
        print(basic_chemical)
        print(precise_quantity_needed)
        print(chemicals[basic_chemical][0])
        print("-"*10)
        ore_cnt += get_required_ore(basic_chemical, precise_quantity_needed, chemicals)
    
    return ore_cnt
    
#     print(basic_chemicals)
#     return get_ore_requirments("FUEL", chemicals)


def parse_raw_input(raw_input):
    chemicals = dict()
    for raw_reaction in raw_input.split("\n"):
        raw_ingredients, raw_target_symbol = raw_reaction.split(" => ")
        target_quantity, target_symbol = parse_raw_symbol(raw_target_symbol)
        
        reaction = list()
        for raw_ingredient in raw_ingredients.split(","):
            ingredient = parse_raw_symbol(raw_ingredient)
            reaction.append(ingredient)
        
        chemicals[target_symbol] = (target_quantity, reaction)
    
    return chemicals

def get_basic_chemicals(chemicals):
    basic_chemicals = list()
    
    for symbol, full_reaction in chemicals.items():
        symbol_quantity, reaction = full_reaction
        
        if len(reaction) > 1:
            continue
            
        quantity, ingredient_symbol = reaction[0]
        
        if ingredient_symbol == "ORE":
            basic_chemicals.append(symbol)
            
    return basic_chemicals

def parse_raw_symbol(raw_symbol):
    raw_quantity, symbol = raw_symbol.strip().split(" ")
    return int(raw_quantity), symbol


def get_required_ore(basic_symbol, reaction_quantity, chemicals_def):
    base_quantity = chemicals_def[basic_symbol][0]
    base_ore_quantity = chemicals_def[basic_symbol][1][0][0]
    
    req_symbol_quantity = base_quantity * math.ceil(reaction_quantity/base_quantity)
    req_ore_quantity = base_ore_quantity * (req_symbol_quantity//base_quantity)
    
    return req_ore_quantity


def get_symbol_quantity(current_symbol, basic_symbol, chemicals_def):
    if current_symbol == basic_symbol:
        return 1
    
    if current_symbol == "ORE":
        return 0

    reaction = chemicals_def[current_symbol][1]
    current_cnt = 0
    
    for ingredient in reaction:
        quantity, symbol = ingredient
        
        current_cnt += quantity * get_symbol_quantity(symbol, basic_symbol, chemicals_def)
        
    return current_cnt

In [4]:
# from advent import *
from collections import defaultdict
import math

with open('inputs/Day_14.txt') as input_file:
    lines = input_file.readlines()


class Formula:
    def __init__(self):
        self.inputs = []
        self.output = None


class ReagentInfo:
    def __init__(self, name, number):
        self.name = name
        self.number = number


in_formulas = defaultdict(list)
out_formulas = {}

ore = 'ORE'
fuel = 'FUEL'

for line in lines:
    inputs, output = line.split("=>")
    output_number, output_name = output.strip().split(" ")

    formula = Formula()
    formula.output = ReagentInfo(output_name, int(output_number))
    for input_reagent in inputs.split(","):
        input_reagent = input_reagent.strip()
        input_number, input_name = input_reagent.split(" ")
        formula.inputs.append(ReagentInfo(input_name, int(input_number)))
        in_formulas[input_name].append(formula)
    out_formulas[output_name] = formula

needed = defaultdict(int)
producing = defaultdict(int)


def produce(name):
    formula = out_formulas[name]
    out_reagent = formula.output
    count = math.ceil(
        max(0, needed[out_reagent.name] - producing[out_reagent.name])
        / out_reagent.number)
    producing[out_reagent.name] += count * out_reagent.number
    for in_reagent in formula.inputs:
        needed[in_reagent.name] += count * in_reagent.number

    for in_reagent in formula.inputs:
        if in_reagent.name != ore:
            produce(in_reagent.name)


needed[fuel] = 1
produce(fuel)
print(f"Part 1 solution {needed[ore]}")

ore_reserve = 1000000000000
fuel_to_produce = 0
fuel_jump = 1
narrowing = False

while True:
    needed.clear()
    producing.clear()
    needed[fuel] = fuel_to_produce
    produce(fuel)
    if needed[ore] > ore_reserve:
        narrowing = True
        fuel_jump = max(1, fuel_jump // 2)
        fuel_to_produce -= fuel_jump
    elif not narrowing:
        fuel_jump *= 2
        fuel_to_produce += fuel_jump
    else:
        if fuel_jump == 1:
            break
        fuel_to_produce += fuel_jump

print(f"Part 2 solution {fuel_to_produce}")

Part 1 solution 1967319
Part 2 solution 1122036
